In [ ]:
import json
import numpy as np

# load the data from the JSON file
with open('/content/sample_data/realistic_trajectory_data.json', 'r') as file:
    data = json.load(file)

trajectories = [[point['x'], point['y']] for point in data]
initial_accs = [[point['ax'], point['ay']] for point in data]
in_goal = [1 if point['in_goal'] else 0 for point in data]
num_timesteps = 13

# prepare the training data
X_train_pos = []
X_train_acc = []
y_train = []
y_train_goal = []

for i in range(len(trajectories) - num_timesteps):
    X_train_pos.append(trajectories[i:i+num_timesteps])
    X_train_acc.append(initial_accs[i])
    y_train.append(trajectories[i + num_timesteps])
    y_train_goal.append(in_goal[i + num_timesteps])

# convert to numpy arrays
X_train_pos = np.array(X_train_pos)
X_train_acc = np.array(X_train_acc)
y_train = np.array(y_train)
y_train_goal = np.array(y_train_goal)


In [ ]:
from keras.models import Model
from keras.layers import LSTM, Dense, Dropout, Input, Concatenate
from keras.regularizers import l2

# define the LSTM model
input_pos = Input(shape=(num_timesteps, 2))  # Only 2 features per timestep
input_acc = Input(shape=(2,))  # Only 2 features for acceleration
# lstm layers
x = LSTM(128, return_sequences=True, kernel_regularizer=l2(0.01))(input_pos)
x = Dropout(0.2)(x)
x = LSTM(128, return_sequences=True, kernel_regularizer=l2(0.01))(x)
x = Dropout(0.2)(x)
x = LSTM(64, kernel_regularizer=l2(0.01))(x)

# combine with acc input
combined = Concatenate(axis=-1)([x, input_acc])
output_pos = Dense(2, kernel_regularizer=l2(0.01), name='position')(combined)  # Only 2 outputs
output_goal = Dense(1, activation='sigmoid', kernel_regularizer=l2(0.01), name='goal')(combined)

model = Model(inputs=[input_pos, input_acc], outputs=[output_pos, output_goal])

# compile the model's output
model.compile(optimizer='adam', loss={'position': 'mean_squared_error', 'goal': 'binary_crossentropy'},
              loss_weights={'position': 1.0, 'goal': 0.1})

# train the model
model.fit([X_train_pos, X_train_acc], [y_train, y_train_goal], epochs=200, batch_size=64, validation_split=0.2)

# save the trained model
model.save('trajectory_goal_model_2D.h5')


Epoch 1/200
82/82 [==============================] - 16s 107ms/step - loss: 103006.5625 - position_loss: 102996.6641 - goal_loss: 57.7351 - val_loss: 67317.0078 - val_position_loss: 67310.3047 - val_goal_loss: 25.3185
Epoch 2/200
82/82 [==============================] - 11s 133ms/step - loss: 44023.9883 - position_loss: 44016.8984 - goal_loss: 29.4795 - val_loss: 27084.5020 - val_position_loss: 27079.0703 - val_goal_loss: 13.1002
Epoch 3/200
82/82 [==============================] - 10s 122ms/step - loss: 16781.3965 - position_loss: 16775.1621 - goal_loss: 21.5255 - val_loss: 9711.4229 - val_position_loss: 9706.4873 - val_goal_loss: 8.9899
Epoch 4/200
82/82 [==============================] - 9s 111ms/step - loss: 5991.0938 - position_loss: 5985.4790 - goal_loss: 16.2741 - val_loss: 3616.1191 - val_position_loss: 3611.5332 - val_goal_loss: 6.4967
Epoch 5/200
82/82 [==============================] - 7s 91ms/step - loss: 2565.8440 - position_loss: 2560.8074 - goal_loss: 11.5356 - val_loss:

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
